88. 合并两个有序数组 

https://leetcode.cn/problems/merge-sorted-array/description/?envType=study-plan-v2&envId=top-interview-150

如下是我最开始的写法，完全没想着一些边界条件和双指针。思路就是，遍历要插入的数组，到合适的地方就插入。

In [1]:
from typing import List

class Solution:
    def merge(self, nums1: List[int], m: int, nums2: List[int], n: int) -> None:
        """
        Do not return anything, modify nums1 in-place instead.
        """
        assert len(nums1) == m + n
        assert len(nums2) == n

        index = 0
        for num in nums2:
            if nums1[index] == 0:
                index += 1
            else:
                while num >= nums1[index] and nums1[index] != 0:
                    index += 1

            nums1 = nums1[:index] + [num] + nums1[index:-1]
        return nums1

这样做的问题如下
- 没有原地修改 nums1，最后一句会创建新列表并把 nums1 这个局部变量指向新对象；LeetCode 判题看的是传入的那个列表对象是否被修改，这样做不会被认为是 in-place；
- 题目说后面 n 个是占位、应忽略，但并没说数组里不会出现 0 这个真实数据；
- 每插入一次都切片拼接：O(m+n)，做 n 次就是 O(n(m+n))，最坏接近 O((m+n)^2)，而标准解是 O(m+n)；
- while num >= nums1[index] and nums1[index] != 0 这里 index += 1 没有边界保护，index 可能跑到 m + n，直接 IndexError。

可以看到问题很多。

正确思路是用双指针。其实这里的特点很明显，如果某题特意留出了一段全 0 的 padding，大概率是要用双指针，其中写入指针的范围就是这片 zero padding。

**指针究竟如何设计？**

可以基于图灵机的想法去理解。存在读写头，因此指针设计就转为了想要读什么、写什么。这道题很明显，想要读的是当前时刻两个数组里哪个数字最大，想要写的就是把这个最大的数写到末尾。这里的一个前提就是倒序处理。

**为什么要倒序处理？什么时候选择倒序？**

只说这里为什么需要，很简单，因为数字已经是从小到大排好序的；如果倒序去读，自然就能按顺序的读到从大到小的数字，和我们的需求相符。因此，i = m - 1 指向 nums1 有效部分末尾，j = n - 1 指向 nums2 末尾，k = m + n - 1 指向 nums1 最末尾，也就是要填的位置。每次把 nums1[i] 和 nums2[j] 中较大的放到 nums1[k]，对应指针左移；最后如果 nums2 还有剩余（j >= 0），继续拷贝进 nums1；如果 nums1 剩余就不用管，因为本来就在那。

In [3]:
class Solution:
    def merge(self, nums1: List[int], m: int, nums2: List[int], n: int) -> None:
        """
        Do not return anything, modify nums1 in-place instead.
        """
        i, j, k = m - 1, n - 1, m + n - 1
        while True:
            if j == -1:
                break
            elif j != -1 and i == -1:
                for _ in range(j+1):
                    nums1[_] = nums2[_]
                break
            else:
                if nums1[i] >= nums2[j]:
                    nums1[k] = nums1[i]
                    k -= 1
                    i -= 1
                else:
                    nums1[k] = nums2[j]
                    k -= 1
                    j -= 1
        

如上是我第二次提交的代码，通过了。最后附上更简洁的实现版本。

In [4]:
from typing import List

class Solution:
    def merge(self, nums1: List[int], m: int, nums2: List[int], n: int) -> None:
        i, j, k = m - 1, n - 1, m + n - 1

        while i >= 0 and j >= 0:
            if nums1[i] > nums2[j]:
                nums1[k] = nums1[i]
                i -= 1
            else:
                nums1[k] = nums2[j]
                j -= 1
            k -= 1

        while j >= 0:
            nums1[k] = nums2[j]
            j -= 1
            k -= 1